In [2]:
import os
from os.path import expanduser
home_dir = expanduser("~")
module_path = home_dir + '/modules/'
import sys
sys.path.append(module_path)
import time
import datetime
import importlib
import random
import tensorflow as tf
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats
import model_management

np.random.seed(999)
random.seed(999)

/opt/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [23]:
### General parameters
run_on_cpu = True
total_set_size = 30000 # how many examples will be used for training+validation+testing
train_size = 10000
val_size = 10000
test_size = 10000
input_features = ['Halo_mass', 'Halo_mass_peak']
output_features = ['Stellar_mass']
nr_iters_before_restart_check = 60 # start making sure that the network did not converge to a local minimum
min_std_tol = 0.01 # minimum allowed std for any parameter
plot_threeD = 0
save_figs = 0
fig_dir = 'figures/'

### Network parameters
nr_hidden_layers = 10 
activationFunction = 'tanh'
nr_neurons_per_layer = 5

### PSO parameters
nIterations = 1000
nParticles = 40
xMin = -10
xMax = 10
alpha = 1
deltaT = 1
c1 = 2
c2 = 2
inertiaWeightStart = 1.4
inertiaWeightMin = 0.3
explorationFraction = 0.8

data_dict = {'X_pos': 0, 'Y_pos': 1, 'Z_pos': 2, 'X_vel': 3, 'Y_vel': 4, 
             'Z_vel': 5, 'Halo_mass': 6, 'Stellar_mass': 7, 'SFR': 8, 
             'Intra_cluster_mass': 9, 'Halo_mass_peak': 10, 
             'Stellar_mass_obs': 11, 'SFR_obs': 12, 'Halo_radius': 13, 
             'Concentration': 14, 'Halo_spin': 15, 'Type': 16}
unit_dict = {'X_pos': '', 'Y_pos': '', 'Z_pos': '', 'X_vel': '', 'Y_vel': '', 
             'Z_vel': '', 'Halo_mass': 'log($M_{G}/M_{S}$)', 'Stellar_mass': 'log($M_{G}/M_{S}$)', 'SFR': '', 
             'Intra_cluster_mass': '', 'Halo_mass_peak': 'log($M_{G}/M_{S}$)', 
             'Stellar_mass_obs': '', 'SFR_obs': '', 'Halo_radius': '', 
             'Concentration': '', 'Halo_spin': '', 'Type': ''}

In [4]:
if run_on_cpu:
    os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [5]:
### Set name ending with parameters for figures to be saved
param_string = 'nLayers_%d_nNeurons_%d_actFun_%s_nTrainSamples_%d_nIterations_%d_' % (
    nr_hidden_layers, nr_neurons_per_layer, activationFunction, train_size, nIterations)
print(param_string)

nLayers_10_nNeurons_10_actFun_tanh_nTrainSamples_1000_nIterations_1000_


# Load and preprocess the data

In [24]:
galfile = pd.read_hdf('/scratch/data/galcats/P200/galaxies.Z01.h5')
galaxies = galfile.as_matrix()
gal_header = galfile.keys().tolist()

### Remove data points with halo mass below 10.5
print(np.shape(galaxies))
galaxies = galaxies[galaxies[:,6] > 10.5, :]
print(np.shape(galaxies))
print(galaxies[:10,6])

halo_min_mass = np.min(galaxies[:, 6])
halo_max_mass = np.max(galaxies[:, 6])

(634742, 17)
(305382, 17)
[11.06128  10.74806  10.517851 11.935673 11.820144 11.564987 10.759135
 10.828308 10.803138 10.55098 ]


In [25]:
### Create the different data sets that will be used
n_data_points = galaxies.shape[0]
subset_indices = np.random.choice(n_data_points, total_set_size, replace=False)
train_indices = subset_indices[: train_size]
val_indices = subset_indices[train_size : train_size+val_size]
test_indices = subset_indices[train_size+val_size :]

x_train = np.zeros((len(train_indices), len(input_features)))
x_val = np.zeros((len(val_indices), len(input_features)))
x_test = np.zeros((len(test_indices), len(input_features)))
y_train = np.zeros((len(train_indices), len(output_features)))
y_val = np.zeros((len(val_indices), len(output_features)))
y_test = np.zeros((len(test_indices), len(output_features)))

for i in range(len(input_features)):
    x_train[:,i] = galaxies[train_indices, data_dict[input_features[i]]]
    x_val[:,i] = galaxies[val_indices, data_dict[input_features[i]]]
    x_test[:,i] = galaxies[test_indices, data_dict[input_features[i]]]
    
for i in range(len(output_features)):
    y_train[:,i] = galaxies[train_indices, data_dict[output_features[i]]]
    y_val[:,i] = galaxies[val_indices, data_dict[output_features[i]]]
    y_test[:,i] = galaxies[test_indices, data_dict[output_features[i]]]

In [26]:
### If you want to preprocess the data

for i in range(np.size(x_train, 1)):
    x_data_means = np.mean(x_train, 0)
    x_data_stds = np.std(x_train, 0)

    x_train_norm = (x_train - x_data_means) / x_data_stds
    x_val_norm = (x_val - x_data_means) / x_data_stds
    x_test_norm = (x_test - x_data_means) / x_data_stds

for i in range(np.size(y_train, 1)):
    y_data_means = np.mean(y_train, 0)
    y_data_stds = np.std(y_train, 0)

    y_train_norm = (y_train - y_data_means) / y_data_stds
    y_val_norm = (y_val - y_data_means) / y_data_stds
    y_test_norm = (y_test - y_data_means) / y_data_stds

In [27]:
print(np.mean(y_test_norm, 0))
print(np.std(y_test_norm, 0))

[0.01384457]
[1.01360266]


In [28]:
### Get a feel for the data
for i in range(len(input_features)):
    print(input_features[i],': min: %.2e, max: %.2e.' % (np.min(x_train[:,i]), np.max(x_train[:,i])))
for i in range(len(output_features)):
    print(output_features[i],': min: %.2e, max: %.2e.' % (np.min(y_train[:,i]), np.max(y_train[:,i])))

Halo_mass : min: 1.05e+01, max: 1.46e+01.
Halo_mass_peak : min: 1.05e+01, max: 1.46e+01.
Stellar_mass : min: 7.00e+00, max: 1.16e+01.


In [ ]:
### Visualisation for when we have 2 input features
%matplotlib notebook
input_feat_1 = 0
input_feat_2 = 1
output_feat = 1

fig = plt.figure(1, figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x_train_norm[:500,input_feat_1], x_train_norm[:500,input_feat_2], 
           y_train_norm[:500,output_feat])
ax.set_xlabel('%s log($M_{H}/M_{S}$)' % (input_features[input_feat_1]))
ax.set_ylabel('%s log($M_{H}/M_{S}$)' % (input_features[input_feat_2]))
ax.set_zlabel('%s log($M_{G}/M_{S}$)' % (output_features[output_feat]))
plt.show()

# Create a new network

In [ ]:
%load_ext autoreload
%autoreload 2

In [29]:
network = Feed_Forward_Neural_Network(nr_hidden_layers, nr_neurons_per_layer, input_features, output_features, 
                                      activationFunction)
network.pso_setup({'inertiaWeightMin': 0.1})
network.pso_train(nIterations, x_train_norm, y_train_norm, x_val_norm, y_val_norm, speed_check=False)


Iteration 0, particle 0, new swarm best. Train: 100.610, Val: 100.349
Iteration 0, particle 1, new swarm best. Train: 40.689, Val: 40.877
Iteration 0, particle 6, new swarm best. Train: 17.866, Val: 18.025
Iteration 0, particle 26, new swarm best. Train: 10.176, Val: 10.312
Iteration 0, particle 27, new swarm best. Train: 5.202, Val: 5.099
Iteration 1, particle 12, new swarm best. Train: 4.111, Val: 4.073
Iteration 2, particle 6, new swarm best. Train: 1.537, Val: 1.522
Iteration 4, particle 10, new swarm best. Train: 1.400, Val: 1.420
Iteration 5, particle 36, new swarm best. Train: 1.014, Val: 1.015
Iteration 7, particle 7, new swarm best. Train: 0.477, Val: 0.469
Iteration 10
Iteration 10, particle 4, new swarm best. Train: 0.366, Val: 0.365
Iteration 17, particle 29, new swarm best. Train: 0.356, Val: 0.349
Iteration 20
Iteration 30
Iteration 40
Iteration 50
Iteration 50, particle 14, new swarm best. Train: 0.327, Val: 0.318
Iteration 52, particle 1, new swarm best. Train: 0.308, V

In [ ]:
### Visualisation of prediction strength for when we have 2 input features
if plot_threeD and len(input_features) == 2:
    predictedY = PredictFunc(bestWeightList, bestBiasList, model, 'test')
    fig = plt.figure(2, figsize=(8,8))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x_test[:,0], x_test[:,1], 
               y_test[:,0], s=3)
    ax.scatter(x_test[:,0], x_test[:,1], 
               predictedY, s=3)
    ax.set_xlabel('%s log($M_{H}/M_{S}$)' % (input_features[0]))
    ax.set_ylabel('%s log($M_{H}/M_{S}$)' % (input_features[1]))
    ax.set_zlabel('%s log($M_{G}/M_{S}$)' % (output_features[0]))

In [ ]:
%matplotlib inline
nBins = 8
bin_edges = np.linspace(halo_min_mass, halo_max_mass, nBins+1)

predictedY = model.predict(x_test)

for i, feat in enumerate(output_features):
    
    
    ### Plot 1
    fig = plt.figure(figsize=(16,16))
    ax = plt.subplot(211)
    plt.plot(y_test[:,i], y_test[:,i], 'k.')
    plt.plot(predictedY[:,i], y_test[:,i], 'g.')
    plt.ylabel('True %s %s' % (feat, unit_dict[feat]), fontsize=15)
    plt.xlabel('Predicted %s %s' % (feat, unit_dict[feat]), fontsize=15)
    plt.legend(['Ideal result', 'predicted ' + feat], loc='upper center')
    plt.title('nIterations: %d, training set size: %d, test mse score: %.2e\n' % (nIterations, 
        train_size, testScore) + 
        '%d input feature(s): [%s]\n%d output feature(s): [%s]\n%d test data points (test) shown' % (
        len(input_features), ', '.join(input_features), len(output_features), ', '.join(output_features),
        test_size), y=1.03, fontsize=20)
    plt.show
        
    if save_figs:
        fig.savefig(fig_dir+'pso_output_scatter_%d_plot_from_' % (i+1)+'_and_'.join(input_features)+'_to_'+
            '_and_'.join(output_features)+'_with_'+param_string+'.png', bbox_inches = 'tight')
    
    ### Plot 2 - boxplot
    
    # bin_means contain (0: mean of the binned values, 1: bin edges, 2: numbers pointing each example to a bin)
    bin_means_true = stats.binned_statistic(x_test[:,i], y_test[:,i], bins=bin_edges)
    bin_means_pred = stats.binned_statistic(x_test[:,i], predictedY[:,i].flatten(), bins=bin_edges)
    bin_centers = []
    for iBin in range(nBins):
        bin_centers.append((bin_means_true[1][iBin] + bin_means_true[1][iBin+1]) / 2)
    sorted_true_y_data = []
    sorted_pred_y_data = []
    for iBin in range(1,nBins+1):
        sorted_true_y_data.append(y_test[bin_means_true[2] == iBin, i])
        sorted_pred_y_data.append(predictedY[bin_means_pred[2] == iBin,i])
    
    fig = plt.figure(figsize=(16,8))
    ax = plt.subplot(212)

    bin_pos = np.array([-2,-1]) # (because this makes it work)
    x_label_centers = []
    for iBin in range(nBins):
        # Every boxplot adds 2 boxes, one from the true data and one from the predicted data
        bin_pos += 3 
        plt.boxplot([sorted_true_y_data[iBin], sorted_pred_y_data[iBin]] , positions = bin_pos, widths = 0.9)
        x_label_centers.append(np.mean(bin_pos))
    
    plt.ylabel('%s %s' % (feat, unit_dict[feat]), fontsize=15)
    plt.xlabel('True Halo mass log($M_{G}/M_{S}$)', fontsize=15)
    ax.set_xlim(left=x_label_centers[0]-2, right=x_label_centers[-1]+2)
    #xlim(0,bin_pos[1] + 1)
    plt.xticks(x_label_centers, bin_centers) TODO fixa siffrorna
    plt.text(12,7,'Left: true data. Right: predicted data.', fontsize=20)
    
    if feat == 'SFR':
        ax.axhline(y=0, linestyle='--')
    
    #plt.title('nIterations: %d, training set size: %d, test mse score: %.2e\n' % (nIterations, 
    #    train_size, testScore) + 
    #    '%d input feature(s): [%s]\n%d output feature(s): [%s]\n%d test data points (test) shown' % (
    #    len(input_features), ', '.join(input_features), len(output_features), ', '.join(output_features),
    #    test_size), y=1.03, fontsize=20)
    
    plt.show()
    
    if save_figs:
        fig.savefig(fig_dir+'pso_output_boxplot_%d_from_' % (i+1)+'_and_'.join(input_features)+'_to_'+
            '_and_'.join(output_features)+'_with_'+param_string+'.png', bbox_inches = 'tight')

In [ ]:
# Summarize history for loss
%matplotlib inline
fig = plt.figure(5, figsize=(8,8))
plt.plot(trainingScoreHistory, 'b')
plt.plot(validationScoreHistory, 'r')
plt.yscale('log')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [19]:
class Feed_Forward_Neural_Network():
    
    def __init__(self, nr_hidden_layers, nr_neurons_per_lay, input_features, output_features, 
                 activation_function):
        
        self.nr_hidden_layers = nr_hidden_layers
        self.nr_neurons_per_lay = nr_neurons_per_lay
        self.input_features = input_features
        self.output_features = output_features
        self.activation_function = activation_function
        
        self.model = None
        
    def pso_setup(self, pso_param_dict={}):
        
        self.pso_swarm = PSO_Swarm(self, self.nr_hidden_layers, self.nr_neurons_per_lay, self.input_features, 
                                   self.output_features, self.activation_function, pso_param_dict=pso_param_dict)
        
    def pso_train(self, nr_iterations, x_train, y_train, x_val, y_val, speed_check=False):
        
        self.pso_swarm.train_network(nr_iterations, x_train, y_train, x_val, y_val, speed_check)


class PSO_Swarm(Feed_Forward_Neural_Network):
    
    def __init__(self, parent, nr_hidden_layers, nr_neurons_per_lay, input_features, output_features, 
                 activation_function, loss_function='mse', metric='mse', pso_param_dict=None):
        self.pso_param_dict = {
            'nr_particles': 40,
            'xMin': -10,
            'xMax': 10,
            'alpha': 1,
            'deltaT': 1,
            'c1': 2,
            'c2': 2,
            'inertiaWeightStart': 1.4,
            'inertiaWeightMin': 0.3,
            'explorationFraction': 0.8,
            'min_std_tol': 0.01
        }
    
        if pso_param_dict is not None:
            for key in pso_param_dict:
                if key in self.pso_param_dict:
                    self.pso_param_dict[key] = pso_param_dict[key]
                else:
                    print('\'%s\ is not a valid key. Choose between:' % (key), self.pso_param_dict.keys())
                    break
        
        self.parent = parent
        self.nr_variables = (nr_hidden_layers-1)*nr_neurons_per_lay**2 + \
            (len(input_features)+len(output_features)+nr_hidden_layers)*nr_neurons_per_lay + len(output_features)
        self.nr_hidden_layers = nr_hidden_layers
        self.nr_neurons_per_lay = nr_neurons_per_lay
        self.activation_function = activation_function
        self.input_features = input_features
        self.output_features = output_features
        self.loss_function = loss_function
        
        self.best_weights = None
        
        self.metric = metric
        self.inertia_weight = self.pso_param_dict['inertiaWeightStart']
        self.vMax = (self.pso_param_dict['xMax']-self.pso_param_dict['xMin']) / self.pso_param_dict['deltaT']
        
        self.set_up_model()
        
        self.initialise_swarm()
        
    def set_up_model(self):
        
        self.model = Sequential()
        self.model.add(Dense(self.nr_neurons_per_lay, input_dim = len(self.input_features), 
                             activation = self.activation_function))
    
        for i in range(0, self.nr_hidden_layers-1):
            self.model.add(Dense(self.nr_neurons_per_lay, activation = self.activation_function))

        self.model.add(Dense(len(self.output_features), activation = None))
                
        self.model.compile(loss=self.loss_function, metrics=[self.metric], optimizer='adam')
        
    def train_network(self, nr_iterations, x_train, y_train, x_val, y_val, speed_check):

        self.nr_iterations_trained = nr_iterations
        self.nr_train_points_used = np.size(x_train, 0)
        self.nr_val_points_used = np.size(x_val, 0)
        
        with open('progress.txt', 'w+') as f:

            # make sure the output isn't just the same
            shouldStartFresh = 1
            while shouldStartFresh:
                shouldStartFresh = 0

                inertia_weight_reduction = np.exp(np.log(self.pso_param_dict['inertiaWeightMin'] / 
                                            self.pso_param_dict['inertiaWeightStart']) / 
                                            self.pso_param_dict['explorationFraction'] / nr_iterations)
                inertia_weight = self.pso_param_dict['inertiaWeightStart']

                self.validationScoreHistory = []
                self.trainingScoreHistory = []
                
                self.avg_speed_before_history = []
                self.avg_speed_after_history = []
                    
                lastTimeSwarmBest = 0
                
                self.initialise_swarm()

                glob_start = time.time()
                for iteration in range(nr_iterations):

                    if (int(iteration/10) == iteration/10) and (iteration > 0):
                        # see if network has run into a local minima                        
                        if (iteration - lastTimeSwarmBest) > nr_iters_before_restart_check:
                            self.set_weights(self.best_weights)
                            y_pred = self.predict_output(x_val_norm)

                            stds = np.std(y_pred, axis=0)
                            print('standard deviations of predicted parameters: ', stds)
                            shouldStartFresh = np.any(stds < self.pso_param_dict['min_std_tol'])
                            if shouldStartFresh:
                                break

                        progress_end = time.time()
                        elapsed_so_far = (progress_end - glob_start) / 60
                        time_remaining = elapsed_so_far / iteration * (self.nr_iterations_trained - iteration)

                        print('Iteration %d' % (iteration))
                        f.write('%s      ' % (datetime.datetime.now().strftime("%H:%M:%S")))
                        f.write('Iterations tried: %d/%d     ' % (iteration, self.nr_iterations_trained))
                        f.write('Elapsed time: %dmin     ' % (elapsed_so_far))
                        f.write('Time remaining: %dmin.\n' % (time_remaining))
                        f.flush()

                    for iParticle, particle in enumerate(self.particle_list):
                        
                        train_score = particle.evaluate_particle(x_train, y_train)

                        is_swarm_best_train = (train_score < self.swarm_best_train)
                        
                        if is_swarm_best_train:
                            
                        
                            lastTimeSwarmBest = iteration
                            self.swarm_best_train = train_score
                            self.swarm_best_position = particle.position
                            
                            val_score = particle.evaluate_particle(x_val, y_val)
                            is_swarm_best_val = (val_score < self.swarm_best_val)
                            if is_swarm_best_val: # only update best weights after val highscore
                                self.best_weights = particle.get_weights()
                            
                            
                            self.validationScoreHistory.append(val_score)
                            self.trainingScoreHistory.append(train_score)

                            print('Iteration %d, particle %d, new swarm best. Train: %.3f, Val: %.3f' % (iteration, 
                                                            iParticle, train_score, val_score))
                            f.write('Iteration %d, particle %d, new swarm best. Train: %.3f, Val: %.3f\n' % (iteration, 
                                                            iParticle, train_score, val_score))
                            f.flush()


                    self.update_swarm(speed_check, f)
                    
                    inertia_weight = self.update_inertia_weight(inertia_weight, inertia_weight_reduction, 
                                                                iteration, f)
                    
                    

        end = time.time()
        
    def update_inertia_weight(self, inertia_weight, inertia_weight_reduction, iteration, f):
        
        isExploring = (inertia_weight > self.pso_param_dict['inertiaWeightMin'])
        if isExploring:
            inertia_weight = inertia_weight * inertia_weight_reduction
            isExploring = (inertia_weight > self.pso_param_dict['inertiaWeightMin'])
            if not isExploring:
                print('SWITCH TO EPLOIT! Iteration %d/%d.' % (iteration, self.nr_iterations_trained))
                f.write('SWITCH TO EPLOIT! Iteration %d/%d.\n' % (iteration, self.nr_iterations_trained))
                f.flush()
        return inertia_weight
        
    def predict_output(self, x_data):
    
        y_pred = self.model.predict(x_data)  # always contains the best model so far

        return y_pred
    
    def initialise_swarm(self):
        
        self.particle_list = []
        
        for i in range(self.pso_param_dict['nr_particles']):
            
            particle = PSO_Particle(self)
            self.particle_list.append(particle)
            
        self.swarm_best_train = 1e20
        self.swarm_best_val = 1e20
        self.swarm_best_position = self.particle_list[0].best_position  # arbitrarily take the first position
        self.best_particle_nr = 0
        
    def update_swarm(self, speed_check, f):
        
        self.speeds_before = []
        self.speeds_after = []
        #self.term_one = []
        #self.term_two = []
        #self.too_fast_count = 0
        #self.mean_particle_best_difference = []
        #self.mean_swarm_best_difference = []
        
        #q = np.random.uniform(size = self.nr_variables)
        #r = np.random.uniform(size = self.nr_variables)
        
        for particle in self.particle_list:
            particle.update_particle()
            
        #print('term 1: ', np.mean(self.term_one))
        #print('term 2:', np.mean(self.term_two)) 
        #print('%d/%d particles were too fast.' % (self.too_fast_count, self.pso_param_dict['nr_particles']))
        #print('mean particle best diff: %.2f'% (np.mean(self.mean_particle_best_difference)))
        #print('mean swarm best diff: %.2f'% (np.mean(self.mean_swarm_best_difference)))
        #print('q: ', np.mean(q))
        #print('r: ', np.mean(r))
        avg_speed_before = np.mean(self.speeds_before)
        avg_speed_after = np.mean(self.speeds_after)
        self.avg_speed_before_history.append(avg_speed_before)
        self.avg_speed_after_history.append(avg_speed_after)
        
        if speed_check:
            print('Average speed of the particles before normalization is: ', avg_speed_before)
            print('Average speed of the particles after normalization is: ', avg_speed_after)
            f.write('Average speed of the particles before normalization is: %.2f' % (avg_speed_before))
            f.write('Average speed of the particles after normalization is: %.2f' % (avg_speed_after))
            f.flush()
            
            
    def set_weights(self, weightList):
        
        weightMatrixList = weightList[0]
        biasList = weightList[1]
        for i in range(len(weightMatrixList)):
            self.model.layers[i].set_weights([weightMatrixList[i], biasList[i]])
            
        
class PSO_Particle(PSO_Swarm):
        
    def __init__(self, parent):
        
        self.parent = parent
            
        r1 = np.random.uniform(size=(self.parent.nr_variables))
        r2 = np.random.uniform(size=(self.parent.nr_variables))

        self.position = self.parent.pso_param_dict['xMin'] + r1 * (self.parent.pso_param_dict['xMax'] - 
                                self.parent.pso_param_dict['xMin'])
        self.velocity = self.parent.pso_param_dict['alpha']/self.parent.pso_param_dict['deltaT'] * \
                        ((self.parent.pso_param_dict['xMin'] - self.parent.pso_param_dict['xMax'])/2 + r2 * 
                         (self.parent.pso_param_dict['xMax'] - self.parent.pso_param_dict['xMin']))
        
        self.best_score = 1e20
        self.best_position = self.position
        
        
        
    def evaluate_particle(self, x_data, y_data):
        
        weightList = self.get_weights()
        self.parent.set_weights(weightList)
        
        score = self.parent.model.evaluate(x_data, y_data, verbose=0)
        if score[0] < self.best_score:
            self.best_score = score[0]
            self.best_position = self.position
            
        return score[0]
        
    def get_weights(self): # sets the weights from the current pos in parameter space
        
        weightMatrixList = [] # will contain a list of all the weight matrices 
        biasList = []   # will contain a list of all the biases

        weightCounter = 0 # to help assign weights and biases to their correct matrix

        ### Extract weight matrices
        input_dim = len(self.parent.input_features)
        output_dim = len(self.parent.output_features)
        weightMatrix = np.zeros((input_dim, self.parent.nr_neurons_per_lay)) 
        for i in range(input_dim):  
            weightMatrix[i,:] = self.position[weightCounter:weightCounter+self.parent.nr_neurons_per_lay]
            weightCounter += self.parent.nr_neurons_per_lay
        weightMatrixList.append(weightMatrix)

        
        for iLayer in range(self.parent.nr_hidden_layers-1):
            weightMatrix = np.zeros((self.parent.nr_neurons_per_lay, self.parent.nr_neurons_per_lay))
            for iNeuron in range(self.parent.nr_neurons_per_lay):

                weightMatrix[iNeuron,:] = self.position[weightCounter:weightCounter+self.parent.nr_neurons_per_lay]
                weightCounter += self.parent.nr_neurons_per_lay

            weightMatrixList.append(weightMatrix)

        weightMatrix = np.zeros((self.parent.nr_neurons_per_lay, output_dim))
        for i in range(self.parent.nr_neurons_per_lay):  
            weightMatrix[i,:] = self.position[weightCounter:weightCounter+output_dim]
            weightCounter += output_dim

        weightMatrixList.append(weightMatrix)

        ### Extract bias vectors
        for iLayer in range(self.parent.nr_hidden_layers):

            biasVector = self.position[weightCounter:weightCounter+self.parent.nr_neurons_per_lay]
            weightCounter += self.parent.nr_neurons_per_lay

            biasList.append(biasVector)

        biasVector = np.zeros(output_dim)
        biasVector = self.position[weightCounter:weightCounter+output_dim] # for the output layer
        biasList.append(biasVector)

        weightCounter += output_dim
        
        weightList = [weightMatrixList, biasList]

        #print(weightCounter == len(self.position))  # a check if the number of variables is correct
        
        return weightList

    def update_particle(self):

        q = np.random.uniform()#size = self.parent.nr_variables)
        r = np.random.uniform()#size = self.parent.nr_variables)
        #print(q)
        #print(r)
        particle_best_difference = self.best_position - self.position
        swarm_best_difference = self.parent.swarm_best_position - self.position
        
        #self.parent.mean_particle_best_difference.append(np.mean(np.abs(particle_best_difference)))
        #self.parent.mean_swarm_best_difference.append(np.mean(np.abs(swarm_best_difference)))

        self.velocity = self.parent.inertia_weight * self.velocity + self.parent.pso_param_dict['c1'] * q * \
                        particle_best_difference / self.parent.pso_param_dict['deltaT'] + \
                        self.parent.pso_param_dict['c2'] * r * swarm_best_difference / \
                        self.parent.pso_param_dict['deltaT']
                    
        #self.parent.term_one.append(np.mean(np.abs(self.parent.pso_param_dict['c1'] * q * \
        #                particle_best_difference / self.parent.pso_param_dict['deltaT'])))
        #self.parent.term_two.append(np.mean(np.abs(self.parent.pso_param_dict['c2'] * r * swarm_best_difference / \
        #                self.parent.pso_param_dict['deltaT'])))

        # now limit velocity to vMax
        absolute_velocity_before_normalization = np.sqrt(np.sum(np.power(self.velocity, 2)))
        is_too_fast = absolute_velocity_before_normalization > self.parent.vMax
        if is_too_fast:
            #self.parent.too_fast_count += 1
            self.velocity = self.velocity * self.parent.vMax / absolute_velocity_before_normalization
            
        absolute_velocity_after_normalization = np.sqrt(np.sum(np.power(self.velocity, 2)))

        self.parent.speeds_before.append(absolute_velocity_before_normalization)
        self.parent.speeds_after.append(absolute_velocity_after_normalization)
            
        self.position = self.position + self.velocity * self.parent.pso_param_dict['deltaT']
        
        
